In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
scenarios = ["4. DeleteBook_NavigateAwayWithoutDeletion_BooksMenu"]

In [4]:
prompt_uc="""Remove the book

Main scenario:
User clicks the "Delete" button (to the right from the book he/she wants to delete) on the book list/details page;
Application displays a dialog page to confirm destroying the record for chosen book;
User confirms deletion by pressing "Confirm" button.

Cancel operation scenario:
User clicks the "Delete" button (to the right from the book he/she wants to delete) on the book list/details page;
Application displays a dialog page to confirm destroying the record for chosen book;
User presses “Cancel” button;
Data doesn’t delete in the database, then a list of books records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be deleted from the database, and the corresponding form with updated data will be opened."""

In [5]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form.
"""

In [6]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            print(gpt_response)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")

```json
{
  "Test Case 1": {
    "testCaseName": "Delete Book and Confirm Deletion",
    "objective": "Verify that a user can delete a book and confirm the deletion process successfully",
    "testSteps": [
      "Step 1: Navigate to the book list/details page.",
      "Step 2: Click the 'Delete' button next to the book intended for deletion.",
      "Step 3: On the confirmation dialog, click the 'Confirm' button."
    ],
    "expectedResult": "The book record should be successfully deleted from the database, and the user should be redirected to the updated list of books without the deleted record.",
    "explanation": "This test case ensures that the deletion process works as intended when a user confirms the deletion. It is directly related to the main scenario of the test scenario provided."
  },
  "Test Case 2": {
    "testCaseName": "Delete Book and Cancel Deletion",
    "objective": "Verify that a user can cancel the deletion of a book after initiating the delete action",
    "te